# **Tarea 5: Modelos de lenguaje neuronales**
##**Procesamiento de lenguaje natural**
## **Alan García Zermeño**
### 18/04/2023

### Usamos el código utilizado para la realización de la práctica 5 y 6 de procesamiento de lenguaje: https://colab.research.google.com/drive/1zvqLxKW7yxbpbh6zUqRk-55HnzR1o-lz?usp=sharing
### Usaremos los siguientes directorios en la nube para realizar el presente trabajo:


*   Datos: https://drive.google.com/drive/folders/1diCrwQVVpXQXNV7iI1CopMPTNvW2kJVv?usp=sharing
*   Modelo de caracteres: https://drive.google.com/drive/folders/1im_mhLNttX47cgULwt14kUh9BfoMoTGj?usp=sharing

*   Embeddings: https://drive.google.com/drive/folders/1o3gRa51E76ZM8DaSW1XZSYE1gJNa6yu2?usp=sharing
*   Modelo con embeddings: https://drive.google.com/drive/folders/1-0Dmtlyjr6TTN6VkcEOYymy6wZA8or7A?usp=sharing
*   Modelo de Bengio: https://drive.google.com/drive/folders/1csiQZspCRISK0yOIJrmsMY2thk-dZbto?usp=sharing













## 1. Realizar modelo de lenguaje basado en caracteres con ngramas de tamaño 6

In [1]:
import os
import time
import shutil
import random
from typing import Tuple
from argparse import Namespace
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score

In [2]:
seed = 1111
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False

In [3]:
X_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mex_data/mex_train.txt',sep = '\r\n',engine = 'python', header = None).loc[:,0].values.tolist()
X_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mex_data/mex_val.txt',sep = '\r\n',engine = 'python', header = None).loc[:,0].values.tolist()
display(X_train[0],X_val[0])

'lo peor de todo es que no me dan por un tiempo y luego vuelven estoy hasta la verga de estl'

'así debería ser siempre para que se mueran a la verga'

In [4]:
args = Namespace()
args.N = 6

### El tokenizador default se ajustó a una función que regresa la lista de texto separada en caracteres.

In [4]:
class NgramData():

  def __init__(self,N: int,vocab_max: int = 5000, tokenizer = None, embeddings_model = None):
    self.tokenizer = tokenizer if tokenizer else self.default_tokenizer
    self.punct = set(['@usuario','<url>','@','*','<','>','"','?','¿','!','¡','»','^','-',':',';','{','}','\xa0','\u200d','«',"¬","°"])
    self.N = N
    self.vocab_max = vocab_max
    self.UNK = "<unk>"
    self.SOS = '<s>'
    self.EOS = "</s>"
    self.embeddings_model = embeddings_model
    if embeddings_model is not None:
      self.embeddings_model_vsize = len(embeddings_model[list(embeddings_model.keys())[0]])
  
  def get_vocab_size(self) -> int:
    return len(self.vocab)

  def default_tokenizer(self, doc:str) -> list:
    return [*doc]
  
  def remove_word(self, word: str) -> bool:
    word = word.lower()
    is_punct = True if word in self.punct else False
    is_digit = word.isnumeric()
    return is_punct or is_digit

  def get_vocab(self, corpus: list) -> set:
    freq_dist = FreqDist([w.lower() for sentence in corpus\
                          for w in self.tokenizer(sentence)\
                          if not self.remove_word(w)])

    sorted_words = self.sortFreqDict(freq_dist)[:self.vocab_max-3]
    return set(sorted_words)
  
  def sortFreqDict(self, freq_dist) -> list:
    freq_dict = dict(freq_dist)
    return sorted(freq_dict, key = freq_dict.get, reverse = True)

  def fit(self, corpus: list) -> None:
    self.vocab = self.get_vocab(corpus)
    self.vocab.add(self.UNK)
    self.vocab.add(self.SOS)
    self.vocab.add(self.EOS)
    self.w2id = {}
    self.id2w = {}

    if self.embeddings_model is not None:
      self.embedding_matrix = np.empty([len(self.vocab), self.embeddings_model_vsize])
    
    id = 0
    for doc in corpus:
      for word in self.tokenizer(doc):
        word_ = word.lower()
        if word_ in self.vocab and not word_ in self.w2id:
          self.w2id[word_] = id
          self.id2w[id] = word_
          
          if self.embeddings_model is not None:
            if word_ in self.embeddings_model:
              self.embedding_matrix[id] = self.embeddings_model[word_]
            else:
              self.embedding_matrix[id] = np.random.rand(self.embeddings_model_vsize)
          
          id+=1
    
    self.w2id.update({self.UNK: id, self.SOS: id+1, self.EOS: id+2})
    self.id2w.update({id: self.UNK, id+1: self.SOS, id+2: self.EOS})

  
  def transform(self, corpus: list) -> Tuple[np.ndarray, np.ndarray]:
    X_ngrams = []
    y = []

    for doc in corpus:
      doc_ngram = self.get_ngram_doc(doc)
      for words_window in doc_ngram:
        words_window_ids = [self.w2id[w] for w in words_window]
        X_ngrams.append(list(words_window_ids[:-1]))
        y.append(words_window_ids[-1])
    
    return np.array(X_ngrams), np.array(y)
  
  def get_ngram_doc(self, doc: str) -> list:
    doc_tokens = self.tokenizer(doc)
    doc_tokens = self.replace_unk(doc_tokens)
    doc_tokens = [w.lower() for w in doc_tokens]
    doc_tokens = [self.SOS]*(self.N-1) + doc_tokens + [self.EOS]
    return list(ngrams(doc_tokens, self.N))
  
  def replace_unk(self, doc_tokens: list) -> list:
    for i, token in enumerate(doc_tokens):
      if token.lower() not in self.vocab:
        doc_tokens[i] = self.UNK
    
    return doc_tokens

In [6]:
tk = TweetTokenizer()
ngram_data = NgramData(args.N,300)#,tk.tokenize)
ngram_data.fit(X_train)

In [7]:
X_ngram_train, y_ngram_train = ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = ngram_data.transform(X_val)

In [9]:
print(f'Training observations: {X_ngram_train.shape}, y: {y_ngram_train.shape}')
print(f'Validation observations: {X_ngram_val.shape}, y: {y_ngram_val.shape}')

Training observations: (498949, 5), y: (498949,)
Validation observations: (54110, 5), y: (54110,)


In [10]:
[[ngram_data.id2w[w] for w in tw] for tw in X_ngram_train[:9]]

[['<s>', '<s>', '<s>', '<s>', '<s>'],
 ['<s>', '<s>', '<s>', '<s>', 'l'],
 ['<s>', '<s>', '<s>', 'l', 'o'],
 ['<s>', '<s>', 'l', 'o', ' '],
 ['<s>', 'l', 'o', ' ', 'p'],
 ['l', 'o', ' ', 'p', 'e'],
 ['o', ' ', 'p', 'e', 'o'],
 [' ', 'p', 'e', 'o', 'r'],
 ['p', 'e', 'o', 'r', ' ']]

In [11]:
[ngram_data.id2w[w] for w  in y_ngram_train[:9]]

['l', 'o', ' ', 'p', 'e', 'o', 'r', ' ', 'd']

In [8]:
args.batch_size = 64
args.num_workers = 2

train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype = torch.int64),
                              torch.tensor(y_ngram_train, dtype = torch.int64))

train_loader = DataLoader(train_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=True)

val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype = torch.int64),
                              torch.tensor(y_ngram_val, dtype = torch.int64))

val_loader = DataLoader(val_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=False)

In [13]:
batch = next(iter(train_loader))
print(f'X shape: {batch[0].shape}')
print(f'y shape: {batch[1].shape}')

X shape: torch.Size([64, 5])
y shape: torch.Size([64])


In [6]:
class NeuralLM(nn.Module):

  def __init__(self, args, embedding_matrix=None):
    super(NeuralLM, self).__init__()

    self.window_size = args.N-1
    self.embedding_dim = args.d

    self.emb = nn.Embedding(args.vocab_size, args.d)
    if embedding_matrix is not None:
      dtp = self.emb.weight.dtype
      self.emb.weight = torch.nn.Parameter(torch.from_numpy(embedding_matrix).to(dtp))
    self.fc1 = nn.Linear(args.d*(args.N-1), args.d_h)
    self.drop1 = nn.Dropout(p=args.dropout)
    self.fc2 = nn.Linear(args.d_h, args.vocab_size, bias=False)
  
  def forward(self, x):
    x = self.emb(x)
    x = x.view(-1, self.window_size*self.embedding_dim)
    h = F.relu(self.fc1(x))
    h = self.drop1(h)
    return self.fc2(h)

In [10]:
def get_preds(raw_logits):
  probs = F.softmax(raw_logits.detach(), dim=1)
  y_pred = torch.argmax(probs, dim=1).cpu().numpy()

  return y_pred

def model_eval(data, model, gpu=False):
  with torch.no_grad():
    preds, tgts = [],[]
    for window_words,labels in data:
      if gpu:
        window_words = window_words.cuda()
      
      outputs = model(window_words)

      #predictions
      y_pred = get_preds(outputs)
      tgt = labels.numpy()
      tgts.append(tgt)
      preds.append(y_pred)

  tgts = [e for l in tgts for e in l]
  preds = [e for l in preds for e in l]
  return accuracy_score(tgts, preds)

def save_checkpoint(state, is_best, checkpoint_path, filename = "checkpoint.pt"):
  filename = os.path.join(checkpoint_path, filename)
  torch.save(state, filename)
  if is_best:
    shutil.copyfile(filename, os.path.join(checkpoint_path, "model_best.pt"))

In [11]:
#Hyperparameters
args.vocab_size = ngram_data.get_vocab_size()
args.d = 128        #dimensión de word embeddings
args.d_h = 256      #HL
args.dropout = 0.1

#Training parameters
args.lr = 2.3e-1
args.num_epochs = 100
args.patience = 20

# Scheduler hyperparameters
args.lr_patience = 10
args.lr_factor = 0.5

#saving directory
args.savedir = '/content/drive/MyDrive/Colab Notebooks/model_char'
os.makedirs(args.savedir, exist_ok=True)

#Create model
model = NeuralLM(args)

#Send to GPU
args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
  model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,"min",
    patience=args.lr_patience,
    verbose=True,
    factor=args.lr_factor
)

In [ ]:
start_time = time.time()
best_metric = 0
metric_history = []
train_metric_history = []

for epoch in range(args.num_epochs):
  epoch_start_time = time.time()
  loss_epoch = []
  training_metric = []
  model.train()

  for window_words, labels in train_loader:

    if args.use_gpu:
      window_words = window_words.cuda()
      labels = labels.cuda()

    outputs = model(window_words)
    loss = criterion(outputs, labels)
    loss_epoch.append(loss.item())

    y_pred = get_preds(outputs)
    tgt = labels.cpu().numpy()
    training_metric.append(accuracy_score(tgt, y_pred))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  mean_epoch_metric = np.mean(training_metric)
  train_metric_history.append(mean_epoch_metric)

  model.eval()
  tuning_metric = model_eval(val_loader, model, gpu=args.use_gpu)
  metric_history.append(mean_epoch_metric)

  scheduler.step(tuning_metric)

  is_improvement = tuning_metric > best_metric
  if is_improvement:
    best_metric = tuning_metric
    n_no_improve = 0

  else:
    n_no_improve += 1
  
  save_checkpoint(
      {
          "epoch": epoch + 1,
          "state_dict": model.state_dict(),
          "optimizer": optimizer.state_dict(),
          "scheduler": scheduler.state_dict(),
          "best_metric": best_metric,
      },
      is_improvement,
      args.savedir,
  )

  if n_no_improve >= args.patience:
    print("No improvement. Breaking out of loop.")
    break

  print('Train acc: {}'.format(mean_epoch_metric))
  print('Epoch [{}/{}], Loss: {:.4f} - Val accuracy: {:.4f} - Epoch time: {:.2f}'
  .format(epoch+1, args.num_epochs, np.mean(loss_epoch), tuning_metric, (time.time()-epoch_start_time)))

print("--- %s seconds ---" %(time.time() - start_time))

In [ ]:
best_model = NeuralLM(args)
best_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/model_char/model_best.pt')['state_dict'])
best_model.train(False);

In [12]:
import tokenize
def parse_text(text, tokenizer):
  #all_tokens = [w.lower() if w in ngram_data.w2id else '<unk>' for w in tokenizer.tokenize(text)]
  all_tokens = [w.lower() if w in ngram_data.w2id else '<unk>' for w in [*text]]
  token_ids = [ngram_data.w2id[word.lower()] for word in all_tokens]
  return all_tokens, token_ids

In [12]:
def sample_next_word(logits, temperature = 1.0):
  logits = np.asarray(logits).astype('float64')
  preds = logits/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probas = np.random.multinomial(1,preds)
  return np.argmax(probas)

def predict_next_token(model, tokens_ids):
  word_ids_tensor = torch.LongTensor(tokens_ids).unsqueeze(0)
  y_raw_pred = model(word_ids_tensor).squeeze(0).detach().numpy()
  y_pred = sample_next_word(y_raw_pred, 1.0)
  return y_pred

def generate_sentence(model, initial_text, tokenizer):
  all_tokens, window_word_ids = parse_text(initial_text, tokenizer)
  for i in range(300):
    y_pred = predict_next_token(best_model, window_word_ids)
    next_word = ngram_data.id2w[y_pred]
    all_tokens.append(next_word)
    if next_word == '</s>':
      break
    else:
      window_word_ids.pop(0)
      window_word_ids.append(y_pred)
  return "".join(all_tokens)

### Cargamos el mejor modelo entrenado y realizamos las pruebas solicitadas. (Borré la salida del entrenamiento previo por accidente, no quise volverla a correr porque tarda mucho en mi cpu)

### Generación de texto:

In [176]:
initial_tokens = "me gu"
print(generate_sentence(best_model, initial_tokens, tk))

me gustas la propedo es que les huebrazos... soy putos de corriet de #felizmiercole vos al vikienar al mandar <unk><unk> es argentincumente por feuso que tienen talle dejarles a mamá estas pero si cambio chingas aconteste cualquiero el mismo quiera mamando. <unk>)</s>


In [195]:
initial_tokens = "no cr"
print(generate_sentence(best_model, initial_tokens, tk))

no cres lo va a faltan notrat de sus hdp hahahh perros</s>


In [25]:
initial_tokens = "soy e"
print(generate_sentence(best_model, initial_tokens, tk))

soy ese en los metodeli te los persona y otra de sería los juegos .️</s>


### Medición de loglikelihood

In [20]:
def log_likelihood(model, text, ngram_model):
  X,y = ngram_data.transform([text])
  X,y = X[2:],y[2:]
  X = torch.LongTensor(X).unsqueeze(0)
  logits = model(X).detach()
  probs = F.softmax(logits, dim = 1).numpy()
  return np.sum([np.log(probs[i][w]) for i,w in enumerate(y)])

In [21]:
frases = ["leo a hegel pero soy materialista","sí estoy de acuerdo compañero ",
          "koolksdjk hb oslskdxxjflskdjf ",
          "chinguen a sus madres cabrones ","dios es mi pastor, nada me falta"]
for ph in frases:
  print("log likelihood de '"+ph+"': ", log_likelihood(best_model, ph,ngram_data))

log likelihood de 'leo a hegel pero soy materialista':  -64.0448
log likelihood de 'sí estoy de acuerdo compañero ':  -43.65916
log likelihood de 'koolksdjk hb oslskdxxjflskdjf ':  -221.7628
log likelihood de 'chinguen a sus madres cabrones ':  -37.46643
log likelihood de 'dios es mi pastor, nada me falta':  -54.523594


### Estructuras Sintácticas

In [39]:
from itertools import permutations
from random import shuffle

word_list = list("chingado")
perm = [''.join(perm) for perm in permutations(word_list)]

for p, t in sorted([(log_likelihood(best_model,text,ngram_data),text) for text in perm], reverse=True)[:5]:
  print(p,''.join(t))
print('-'*50)
for p, t in sorted([(log_likelihood(best_model,text,ngram_data),text) for text in perm], reverse=True)[-5:]:
  print(p,''.join(t))

-8.226238 chingado
-12.024684 ghicando
-12.834872 dgichona
-12.950214 chingoda
-13.811747 hgnicado
--------------------------------------------------
-80.1418 cahgdnoi
-80.60216 iadhnocg
-80.69323 hdoaicng
-83.08769 haocgdni
-83.404396 nahgdcoi


### Medición de perplejidad sobre los datos de validación.

In [22]:
def Prpx(N,p):
  return(np.power(2,-(1/N)*p))

In [57]:
text = ' '.join(X_val)
Prpx(len(text),log_likelihood(best_model,text,ngram_data))

3.10855247056122

## 2. Realizamos el mismo modelo que en la práctica pero cargando los embeddings proporcionados word2vec con ngramas de tamaño 3.

In [5]:
with open('/content/drive/MyDrive/Colab Notebooks/word2vec/word2vec_col.txt','r') as f:
  embw = [line.split() for line in f]
embd = {}
for i in range(1,len(embw)): embd[embw[i][0]] = [float(x) for x in embw[i][1:]]

In [8]:
tk = TweetTokenizer()
args = Namespace()
args.N = 4
ngram_data = NgramData(args.N,5000,tk.tokenize,embeddings_model=embd)
ngram_data.fit(X_train)
X_ngram_train, y_ngram_train = ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = ngram_data.transform(X_val)
del embd

In [9]:
args.batch_size = 64
args.num_workers = 2

train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype = torch.int64),
                              torch.tensor(y_ngram_train, dtype = torch.int64))
train_loader = DataLoader(train_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=True)
val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype = torch.int64),
                              torch.tensor(y_ngram_val, dtype = torch.int64))
val_loader = DataLoader(val_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=False)

In [10]:
args.vocab_size = ngram_data.get_vocab_size()
args.d = 100
args.d_h = 256
args.dropout = 0.1
args.lr = 1e-2
args.num_epochs = 200
args.patience = 20
args.lr_patience = 10
args.lr_factor = 0.5

args.savedir = '/content/drive/MyDrive/Colab Notebooks/nmodel'
os.makedirs(args.savedir, exist_ok=True)
criterion = nn.CrossEntropyLoss()

In [11]:
model = NeuralLM(args,ngram_data.embedding_matrix)
args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,"min",
    patience=args.lr_patience,
    verbose=True,
    factor=args.lr_factor
)

In [80]:
start_time = time.time()
best_metric = 0
metric_history = []
train_metric_history = []

for epoch in range(args.num_epochs):
  epoch_start_time = time.time()
  loss_epoch = []
  training_metric = []
  model.train()

  for window_words, labels in train_loader:

    if args.use_gpu:
      window_words = window_words.cuda()
      labels = labels.cuda()

    outputs = model(window_words)
    loss = criterion(outputs, labels)
    loss_epoch.append(loss.item())

    y_pred = get_preds(outputs)
    tgt = labels.cpu().numpy()
    training_metric.append(accuracy_score(tgt, y_pred))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  mean_epoch_metric = np.mean(training_metric)
  train_metric_history.append(mean_epoch_metric)

  model.eval()
  tuning_metric = model_eval(val_loader, model, gpu=args.use_gpu)
  metric_history.append(mean_epoch_metric)

  scheduler.step(tuning_metric)

  is_improvement = tuning_metric > best_metric
  if is_improvement:
    best_metric = tuning_metric
    n_no_improve = 0

  else:
    n_no_improve += 1
  
  save_checkpoint(
      {
          "epoch": epoch + 1,
          "state_dict": model.state_dict(),
          "optimizer": optimizer.state_dict(),
          "scheduler": scheduler.state_dict(),
          "best_metric": best_metric,
      },
      is_improvement,
      args.savedir,
  )

  if n_no_improve >= args.patience:
    print("No improvement. Breaking out of loop.")
    break

  print('Train acc: {}'.format(mean_epoch_metric))
  print('Epoch [{}/{}], Loss: {:.4f} - Val accuracy: {:.4f} - Epoch time: {:.2f}'
  .format(epoch+1, args.num_epochs, np.mean(loss_epoch), tuning_metric, (time.time()-epoch_start_time)))

print("--- %s seconds ---" %(time.time() - start_time))

Train acc: 0.1651429110399917
Epoch [1/200], Loss: 5.6515 - Val accuracy: 0.2110 - Epoch time: 43.01
Train acc: 0.181641997111857
Epoch [2/200], Loss: 5.3284 - Val accuracy: 0.2110 - Epoch time: 45.02
Train acc: 0.1879236268311087
Epoch [3/200], Loss: 5.1417 - Val accuracy: 0.2133 - Epoch time: 42.67
Train acc: 0.1920306117138917
Epoch [4/200], Loss: 5.0335 - Val accuracy: 0.2163 - Epoch time: 41.58
Train acc: 0.19497241953529518
Epoch [5/200], Loss: 4.9534 - Val accuracy: 0.2200 - Epoch time: 41.91
Train acc: 0.1969189888897562
Epoch [6/200], Loss: 4.8874 - Val accuracy: 0.2224 - Epoch time: 41.54
Train acc: 0.1991859209533474
Epoch [7/200], Loss: 4.8296 - Val accuracy: 0.2199 - Epoch time: 41.32
Train acc: 0.2004202117971535
Epoch [8/200], Loss: 4.7799 - Val accuracy: 0.2239 - Epoch time: 42.24
Train acc: 0.20347219511877812
Epoch [9/200], Loss: 4.7319 - Val accuracy: 0.2236 - Epoch time: 45.24
Train acc: 0.20468006010459763
Epoch [10/200], Loss: 4.6913 - Val accuracy: 0.2227 - Epoch

In [12]:
best_model = NeuralLM(args)
best_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/nmodel/model_best.pt')['state_dict'])
best_model.train(False);

In [13]:
import tokenize
def parse_text(text, tokenizer):
  all_tokens = [w.lower() if w in ngram_data.w2id else '<unk>' for w in tokenizer.tokenize(text)]
  token_ids = [ngram_data.w2id[word.lower()] for word in all_tokens]
  return all_tokens, token_ids

def sample_next_word(logits, temperature = 1.0):
  logits = np.asarray(logits).astype('float64')
  preds = logits/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probas = np.random.multinomial(1,preds)
  return np.argmax(probas)

def predict_next_token(model, tokens_ids):
  word_ids_tensor = torch.LongTensor(tokens_ids).unsqueeze(0)
  y_raw_pred = model(word_ids_tensor).squeeze(0).detach().numpy()
  y_pred = sample_next_word(y_raw_pred, 1.0)
  return y_pred

def generate_sentence(model, initial_text, tokenizer):
  all_tokens, window_word_ids = parse_text(initial_text, tokenizer)
  for i in range(100):
    y_pred = predict_next_token(best_model, window_word_ids)
    next_word = ngram_data.id2w[y_pred]
    all_tokens.append(next_word)

    if next_word == '</s>':
      break
    else:
      window_word_ids.pop(0)
      window_word_ids.append(y_pred)
  return " ".join(all_tokens)

### Cargamos el mejor modelo y realizamos las pruebas solicitadas.

### Palabras más cercanas usando los embeddings cargados.

In [13]:
def print_closest_words(embeddings, ngram_data, word, n):
  word_id = torch.LongTensor([ngram_data.w2id[word]]) #get word id
  word_embed = embeddings(word_id) #get word embedding
  dists = torch.norm(embeddings.weight - word_embed, dim=1).detach()
  lst = sorted(enumerate(dists.numpy()), key = lambda x: x[1]) #sort by distance
  for idx, difference in lst[1:n+1]:
    print(ngram_data.id2w[idx],difference)

In [14]:
print_closest_words(best_model.emb, ngram_data, "perro", 10)

gato 9.554484
perrito 12.496251
gatito 15.656715
enano 17.228447
gordo 17.415058
vecino 17.43069
niño 17.560242
macho 17.807663
sapo 17.827457
mono 17.899689


In [33]:
print_closest_words(best_model.emb, ngram_data, "pendejo", 10)

imbécil 9.731764
tonto 11.607184
idiota 11.913824
tarado 12.543361
cabrón 13.262273
maricon 13.555503
marica 13.810448
mamón 13.912109
maricón 14.1301985
cabron 14.339897


In [42]:
print_closest_words(best_model.emb, ngram_data, "youtuber", 10)

tuitero 13.398776
pendejazo 15.068455
puberto 15.095328
dalas 15.3225565
profesionista 15.408114
meme 15.463341
jotito 15.641256
reportero 15.663945
fake 15.705067
mayate 15.72003


### Generación de texto

In [30]:
initial_tokens = "<s> hola amigo"
print(generate_sentence(best_model, initial_tokens, tk))

<s> hola amigo tiene mil putas doble <unk> <unk> pedas v a todos a supe que nos vale . </s>


In [17]:
initial_tokens = "soy una persona"
print(generate_sentence(best_model, initial_tokens, tk))

soy una persona mejor día del mejor ahora es <unk> pero todo <unk> lo que #putita porque era playera de mujeres no se vuelva a mundial y todos los <unk> 😻 yo gay si no sabe ( qué le caga el <unk> kilos de su puta madre </s>


In [19]:
initial_tokens = "yo creo que"
print(generate_sentence(best_model, initial_tokens, tk))

yo creo que quiero escuchar perder el día de cuenta porque dos los demás a la 💕 saliste <unk> <unk> </s>


### Medición de loglikelihood

In [21]:
frases = ["leo a hegel pero soy materialista","sí estoy de acuerdo compañero ",
          "koolksdjk hb oslskdxxjflskdjf ",
          "chinguen a sus madres cabrones ","dios es mi pastor, nada me falta"]
for ph in frases:
  print("log likelihood de '"+ph+"': ", log_likelihood(best_model, ph,ngram_data))

log likelihood de 'leo a hegel pero soy materialista':  -18.066015
log likelihood de 'sí estoy de acuerdo compañero ':  -29.875872
log likelihood de 'koolksdjk hb oslskdxxjflskdjf ':  -3.381772
log likelihood de 'chinguen a sus madres cabrones ':  -24.233398
log likelihood de 'dios es mi pastor, nada me falta':  -27.192327


### Estructuras sintácticas

In [25]:
from itertools import permutations
from random import shuffle

word_list = "leo messi es campeón del mundo".split(' ')
perm = [' '.join(perm) for perm in permutations(word_list)]

for p, t in sorted([(log_likelihood(best_model,text,ngram_data),text) for text in perm], reverse=True)[:5]:
  print(p,t)
print('-'*50)
for p, t in sorted([(log_likelihood(best_model,text,ngram_data),text) for text in perm], reverse=True)[-5:]:
  print(p,t)

-24.187115 leo campeón del mundo es messi
-25.334713 leo messi es campeón del mundo
-25.383858 messi campeón del mundo es leo
-25.441917 messi campeón es del mundo leo
-25.717482 campeón messi del mundo es leo
--------------------------------------------------
-48.267155 del es leo campeón mundo messi
-48.509388 messi es leo campeón mundo del
-48.592392 messi del es leo campeón mundo
-48.78185 del es messi campeón leo mundo
-49.08704 es leo campeón mundo messi del


### Perplejidad del modelo

In [14]:
def log_likelihood(model, text, ngram_model):
  X,y = ngram_data.transform([text])
  n = X.shape[0]
  X,y = X[2:],y[2:]
  X = torch.LongTensor(X).unsqueeze(0)
  logits = model(X).detach()
  probs = F.softmax(logits, dim = 1).numpy()
  return n,np.sum([np.log(probs[i][w]) for i,w in enumerate(y)])

def Prpx(N,p):
  return(np.power(2,-(1/N)*p))

In [16]:
N,P = 0,0
for i in X_val:
  n,p = log_likelihood(best_model,i,ngram_data)
  N += n
  P += p
Prpx(N,P)

16.368275236521857

### Usando los embeddings word2vec, alcanzamos una precisión de hasta 0.225 sobre los datos de entrenamiento y de casi 0.23 sobre los de validación. Mientras que la perplejidad (16.368) es menor que la calculada para el modelo realizada en clase sin embeddings (20.01).

## 3. Modelo de ngramas de tamaño 3 con conexión directa de la capa de embeddings hacía la salida.

In [6]:
class NeuralLM(nn.Module):
  def __init__(self, args, embedding_matrix=None):
    super(NeuralLM, self).__init__()

    self.window_size = args.N-1
    self.embedding_dim = args.d

    self.emb = nn.Embedding(args.vocab_size, args.d)
    if embedding_matrix is not None:
      dtp = self.emb.weight.dtype
      self.emb.weight = torch.nn.Parameter(torch.from_numpy(embedding_matrix).to(dtp))
    
    self.fcC = nn.Linear(args.d*(args.N-1), args.vocab_size)
    self.fc1 = nn.Linear(args.d*(args.N-1), args.d_h)
    self.drop1 = nn.Dropout(p=args.dropout)
    self.fc2 = nn.Linear(args.d_h, args.vocab_size, bias=False)
  
  def forward(self, x):
    x = self.emb(x)
    x = x.view(-1, self.window_size*self.embedding_dim)
    h = F.relu(self.fc1(x))
    h = self.drop1(h)
    theta = self.fcC(x)
    return self.fc2(h)+theta

In [7]:
tk = TweetTokenizer()
args = Namespace()
args.N = 4
ngram_data = NgramData(args.N,5000,tk.tokenize,embeddings_model=embd)
ngram_data.fit(X_train)
X_ngram_train, y_ngram_train = ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = ngram_data.transform(X_val)
del embd

In [8]:
args.batch_size = 64
args.num_workers = 2

train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype = torch.int64),
                              torch.tensor(y_ngram_train, dtype = torch.int64))
train_loader = DataLoader(train_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=True)
val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype = torch.int64),
                              torch.tensor(y_ngram_val, dtype = torch.int64))
val_loader = DataLoader(val_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=False)

In [9]:
args.vocab_size = ngram_data.get_vocab_size()
args.d = 100
args.d_h = 256
args.dropout = 0.1
args.lr = 1e-2
args.num_epochs = 100
args.patience = 20
args.lr_patience = 10
args.lr_factor = 0.5

args.savedir = '/content/drive/MyDrive/Colab Notebooks/bengio_model'
os.makedirs(args.savedir, exist_ok=True)
criterion = nn.CrossEntropyLoss()

model = NeuralLM(args,ngram_data.embedding_matrix)
args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,"min",
    patience=args.lr_patience,
    verbose=True,
    factor=args.lr_factor
)

In [26]:
start_time = time.time()
best_metric = 0
metric_history = []
train_metric_history = []

for epoch in range(args.num_epochs):
  epoch_start_time = time.time()
  loss_epoch = []
  training_metric = []
  model.train()

  for window_words, labels in train_loader:

    if args.use_gpu:
      window_words = window_words.cuda()
      labels = labels.cuda()

    outputs = model(window_words)
    loss = criterion(outputs, labels)
    loss_epoch.append(loss.item())

    y_pred = get_preds(outputs)
    tgt = labels.cpu().numpy()
    training_metric.append(accuracy_score(tgt, y_pred))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  mean_epoch_metric = np.mean(training_metric)
  train_metric_history.append(mean_epoch_metric)

  model.eval()
  tuning_metric = model_eval(val_loader, model, gpu=args.use_gpu)
  metric_history.append(mean_epoch_metric)

  scheduler.step(tuning_metric)

  is_improvement = tuning_metric > best_metric
  if is_improvement:
    best_metric = tuning_metric
    n_no_improve = 0

  else:
    n_no_improve += 1
  
  save_checkpoint(
      {
          "epoch": epoch + 1,
          "state_dict": model.state_dict(),
          "optimizer": optimizer.state_dict(),
          "scheduler": scheduler.state_dict(),
          "best_metric": best_metric,
      },
      is_improvement,
      args.savedir,
  )

  if n_no_improve >= args.patience:
    print("No improvement. Breaking out of loop.")
    break

  print('Train acc: {}'.format(mean_epoch_metric))
  print('Epoch [{}/{}], Loss: {:.4f} - Val accuracy: {:.4f} - Epoch time: {:.2f}'
  .format(epoch+1, args.num_epochs, np.mean(loss_epoch), tuning_metric, (time.time()-epoch_start_time)))

print("--- %s seconds ---" %(time.time() - start_time))

Train acc: 0.15728345432310775
Epoch [1/100], Loss: 6.4091 - Val accuracy: 0.1973 - Epoch time: 69.47
Train acc: 0.18265918936851144
Epoch [2/100], Loss: 5.5580 - Val accuracy: 0.2137 - Epoch time: 67.30
Train acc: 0.18889040668175786
Epoch [3/100], Loss: 5.1776 - Val accuracy: 0.2102 - Epoch time: 69.49
Train acc: 0.19173504865610283
Epoch [4/100], Loss: 4.9315 - Val accuracy: 0.2125 - Epoch time: 67.85
Train acc: 0.19671002133583115
Epoch [5/100], Loss: 4.7724 - Val accuracy: 0.1960 - Epoch time: 68.66
Train acc: 0.20011692425779928
Epoch [6/100], Loss: 4.6467 - Val accuracy: 0.2188 - Epoch time: 66.60
Train acc: 0.20296359899045094
Epoch [7/100], Loss: 4.5422 - Val accuracy: 0.2169 - Epoch time: 69.35
Train acc: 0.20515694520334088
Epoch [8/100], Loss: 4.4517 - Val accuracy: 0.2194 - Epoch time: 67.22
Train acc: 0.20870614120677544
Epoch [9/100], Loss: 4.3699 - Val accuracy: 0.2170 - Epoch time: 69.60
Train acc: 0.2107787415762496
Epoch [10/100], Loss: 4.2956 - Val accuracy: 0.2112 

In [10]:
best_model = NeuralLM(args)
best_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/bengio_model/model_best.pt')['state_dict'])
best_model.train(False);

In [11]:
def log_likelihood(model, text, ngram_model):
  X,y = ngram_data.transform([text])
  n = X.shape[0]
  X,y = X[2:],y[2:]
  X = torch.LongTensor(X).unsqueeze(0)
  logits = model(X).detach()
  probs = F.softmax(logits, dim = 1).numpy()
  return n,np.sum([np.log(probs[i][w]) for i,w in enumerate(y)])

def Prpx(N,p):
  return(np.power(2,-(1/N)*p))

In [20]:
frases = ["leo a hegel pero soy materialista","sí estoy de acuerdo compañero ",
          "koolksdjk hb oslskdxxjflskdjf ",
          "chinguen a sus madres cabrones ","dios es mi pastor, nada me falta"]
for ph in frases:
  print("log likelihood de '"+ph+"': ", log_likelihood(best_model, ph,ngram_data))

log likelihood de 'leo a hegel pero soy materialista':  (7, -19.083868)
log likelihood de 'sí estoy de acuerdo compañero ':  (6, -30.44687)
log likelihood de 'koolksdjk hb oslskdxxjflskdjf ':  (4, -3.1049876)
log likelihood de 'chinguen a sus madres cabrones ':  (6, -28.797634)
log likelihood de 'dios es mi pastor, nada me falta':  (9, -28.738453)


### Se carga el mejor modelo y se mide la perplejidad

In [12]:
N,P = 0,0
for i in X_val:
  n,p = log_likelihood(best_model,i,ngram_data)
  N += n
  P += p
Prpx(N,P)

17.663239640306976

### El entrenamiento alcanzó cerca del 0.27 de precisión sobre los datos de entrenamiento y cerca de 0.23 sobre los de validación, resultados mejores que los del modelo hecho en clase, así como la perplejidad (17.66) fue mejor que la del modelo realizado en clase (20.01).